In [2]:
from scapy.all import *
import pandas as pd 
import numpy as np 
import binascii
import seaborn as sns
sns.set(color_codes = True)
%matplotlib inline
from scapy.layers.inet import *

# Laboratorio 5
### Ricardo Valenzuela - Sara Zavala

## Cargar archivo pcap

In [7]:
pcap_file = "analisis_paquetes.pcap"

In [8]:
info = rdpcap(pcap_file)

### Capturar 100 paquetes

In [9]:
amount_of_packets = 100
pcap = sniff(count = amount_of_packets)
print("Tipo pcap: ",type(pcap))
print("Largo pcap: ", len(pcap))
print(pcap)

Tipo pcap:  <class 'scapy.plist.PacketList'>
Largo pcap:  100
<Sniffed: TCP:67 UDP:29 ICMP:0 Other:4>


### Añada al pcap de 100 paquetes el archivo analisis_paquetes.pcap 

In [10]:
pcap = pcap + info
print(pcap)

<Sniffed+analisis_paquetes.pcap: TCP:67 UDP:91 ICMP:0 Other:4>


### Convierta el pcap a un DataFrame 

In [14]:
ip_fields = [field.name for field in IP().fields_desc]
tcp_fields = [field.name for field in TCP().fields_desc]
udp_fields = [field.name for field in UDP().fields_desc]

dataframe_fields = ip_fields + ["time"] + tcp_fields + ['payload','payload_raw','payload_hex']

df = pd.DataFrame(columns = dataframe_fields)

for packet in pcap[IP]:
    field_values = []
    for field in ip_fields:
        if field == "options":
            field_values.append(len(packet[IP].fields[field]))
        else:
            field_values.append(packet[IP].fields[field])
    field_values.append(packet.time)
    layer_type = type(packet[IP].payload)
    for field in tcp_fields:
        try:
            if field =="options":
                field_values.append(len(packet[layer_type].fields[field]))
            else:
                field_values.append(packet[layer_type].fields[field])
        except:
            field_values.append(None)
    
    field_values.append(len(packet[layer_type].payload))
    field_values.append(packet[layer_type].payload.original)
    field_values.append(binascii.hexlify(packet[layer_type].payload.original))
    
    df_append = pd.DataFrame([field_values], columns=dataframe_fields)
    df = pd.concat([df, df_append], axis=0)


In [17]:
# pcap_file = rdpcap('analisis_paquetes.pcap')
# df = create_dataframe(pcap_file)

df = df.reset_index()
df = df.drop(columns="index")
df.head(10)

,version,ihl,tos,len,id,flags,frag,ttl,proto,chksum,...,dataofs,reserved,flags,window,chksum,urgptr,options,payload,payload_raw,payload_hex
0,4,5,184,257,788,DF,0,64,6,0,...,5,0,PA,33,13670,0,0,217,b'\x17\x03\x03\x00\xd4\r\xd8\xd2[\xf4\xc2>\xf9...,b'17030300d40dd8d25bf4c23ef91ba044aab7c7ea61ab...
1,4,5,0,40,15920,DF,0,48,6,37138,...,5,0,A,27,40176,0,None,0,b'',b''
2,4,5,0,235,15921,DF,0,48,6,36942,...,5,0,PA,27,40339,0,0,195,b'\x17\x03\x03\x00\xbe\x15kn\x1d\xaf\x00\xd7\x...,b'17030300be156b6e1daf00d7956cb1c2254d486d7a36...
3,4,5,0,77,24181,,0,64,17,0,...,None,None,None,None,33755,None,None,49,b'L6\x01\x00\x00\x01\x00\x00\x00\x00\x00\x00\x...,b'4c360100000100000000000010636f6e74656e742d61...
4,4,5,184,40,789,DF,0,64,6,0,...,5,0,A,32,39976,0,None,0,b'',b''
5,4,5,0,93,0,DF,0,64,17,46847,...,None,None,None,None,24533,None,None,65,b'L6\x81\x80\x00\x01\x00\x01\x00\x00\x00\x00\x...,b'4c368180000100010000000010636f6e74656e742d61...
6,4,5,184,1278,43569,DF,0,64,17,0,...,None,None,None,None,28958,None,None,1250,b'\xc5\x00\x00\x00\x01\x08\x87E\x0f\xfd\xcd\x9...,b'c5000000010887450ffdcd99d880004046004493d2e5...
7,4,5,0,102,43570,DF,0,64,17,0,...,None,None,None,None,3792,None,None,74,"b""\xd4\x00\x00\x00\x01\x08\x87E\x0f\xfd\xcd\x9...",b'd4000000010887450ffdcd99d880004039ea1dbfe491...
8,4,5,0,567,43571,DF,0,64,17,0,...,None,None,None,None,4257,None,None,539,b'\xd2\x00\x00\x00\x01\x08\x87E\x0f\xfd\xcd\x9...,b'd2000000010887450ffdcd99d88000420ae354de470a...
9,4,5,0,1278,0,DF,0,59,17,11395,...,None,None,None,None,36532,None,None,1250,b'\xc5\x00\x00\x00\x01\x00\x08\x87E\x0f\xfd\xc...,b'c500000001000887450ffdcd99d88000438f124f4422...


### Muestre los valores de las columnas: Src Addres, Dst Address, Src Port y Dst Port 

In [18]:
df[['src','dst','sport','dport']]

,src,dst,sport,dport
0,192.168.1.63,170.114.15.52,4139,443
1,170.114.15.52,192.168.1.63,443,4139
2,170.114.15.52,192.168.1.63,443,4139
3,192.168.1.63,192.168.1.1,56236,53
4,192.168.1.63,170.114.15.52,4139,443
...,...,...,...,...
153,84.54.22.33,10.1.10.53,53,53
154,10.1.10.53,84.54.22.33,53,53
155,84.54.22.33,10.1.10.53,53,53
156,10.1.10.53,84.54.22.33,53,53


### Estadisticas

#### Muestre cual es la IP origen que más aparece 

In [19]:
frequent_src_address = df['src'].describe()['top']
print(frequent_src_address)

192.168.1.63


#### Muestre cual es la IP destino que más aparece 

In [20]:
frequent_dst_address = df['dst'].describe()['top']
print(frequent_dst_address)

192.168.1.63


#### ¿A qué IPs se comunica la IP del inciso a? 

In [21]:
print(df[df['src'] == frequent_src_address]['dst'].unique())

['170.114.15.52' '192.168.1.1' '142.250.189.138' '192.168.1.255'
 '151.101.4.193' '151.101.65.69' '151.101.129.69' '34.224.19.92'
 '149.154.175.51' '142.250.217.196' '142.250.217.174' '140.82.112.26'
 '239.255.255.250']


#### ¿A qué puertos destino se comunica la IP del inciso a? 

In [22]:
print(df[df['src'] == frequent_src_address]['dport'].unique())

[443 53 51007 1900]


#### ¿A qué puertos origen se comunica la IP del inciso a? 

In [23]:
print(df[df['src'] == frequent_src_address]['sport'].unique())

[4139 56236 56402 49227 4490 4486 4488 4572 4258 4574 4575 4580 4316 60054
 4581 4582]


#### ¿Para que se utiliza el puerto que más aparece en el inciso  d y e? 

Se utiliza para el protocolo TCP. Se necesita que sea origen y destino